In [ ]:
import mushare as ts  
import datetime  
import os  
  
# 设置Tushare Pro的token，需要先在Tushare Pro网站上申请一个token  
ts.set_token('20231204122454-0c0a95da-2b53-412d-9837-d442f49cf70d')
  
# 初始化pro接口  
pro = ts.pro_api()  

In [ ]:
import concurrent.futures

# Set start and end dates for one year period
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=365)
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')

# Get all stock codes
stocks = pro.stock_basic(exchange='', list_status='L', fields='ts_code')

# Main directory for data storage
main_dir = 'a_share_data'
os.makedirs(main_dir, exist_ok=True)

def download_stock_data(ts_code):
    stock_dir = os.path.join(main_dir, ts_code)
    os.makedirs(stock_dir, exist_ok=True)  # Create a directory for each stock

    print(f"Downloading 1min data for {ts_code}...")

    # Download 1min data using pro_bar interface
    df = ts.pro_bar(ts_code=ts_code, asset='E', start_date=start_date, end_date=end_date, freq='1min')

    if df is not None and not df.empty:
        file_path = os.path.join(stock_dir, '1min.csv')
        df.to_csv(file_path, index=False)
        print(f"Saved 1min data for {ts_code} to {file_path}")

# Using ThreadPoolExecutor to download data for each stock in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Adjust max_workers based on your system's capabilities and API's rate limits
    executor.map(download_stock_data, stocks['ts_code'])
